In [ ]:
import pygplates
import time as tme
import numpy as np
import pyvista as pv
from numba import jit
import matplotlib.pyplot as plt
from scipy.spatial import cKDTree
from CodeAfterFirstNotebook import Earth

In the previous notebook, we discussed how to move tectonic plates along a spherical earth. We will now discuss how we approximate various tectonic phenomina onto the earth, which will involve getting the distance and speeds that vertices are from the plate boundaries.

# Plate Boundaries

In calculating various tectonic forces, we will be needing data about various types of plate boundaries. We create a general *PlateBoundary* class, and two child-classes *SubductionBoundary* and *RidgeBoundary*. To initiate plate boundary, we will pass the sharedBound object obtained from the *pygplates* library, which provides coordinates and other properties about plate boundaries.

In [ ]:
#Define some parameters
platePolygonsDirectory = './dataPygplates/Matthews_etal_GPC_2016_MesozoicCenozoic_PlateTopologies_PMAG.gpmlz'
rotationsDirectory = './dataPygplates/Matthews_etal_GPC_2016_410-0Ma_GK07_PMAG.rot'
rotationModel = pygplates.RotationModel(rotationsDirectory)
earthRadius = 6378.137
time = 0

#Create a general plate boundary class
class PlateBoundary:
    def __init__(self, sharedBound):
        
        #Extract relevant data from sharedBounds
        lon, lat, sharedPlateIds = [], [], []
        for sharedSubSection in sharedBound.get_shared_sub_segments():
            latLon = sharedSubSection.get_resolved_geometry().to_lat_lon_array()
            sharedId = [i.get_resolved_feature().get_reconstruction_plate_id() for i in 
                                        sharedSubSection.get_sharing_resolved_topologies()]
            for i in range(len(latLon)):
                lon.append(latLon[i, 1])
                lat.append(latLon[i, 0])
                sharedPlateIds.append(sharedId)
            
            #for i in sharedSubSection.get_sharing_resolved_topologies():
            #    sharedPlateIds.append(i.get_resolved_feature().get_reconstruction_plate_id())
        
        #Store data as class attributes
        self.lon = np.array(lon)
        self.lat = np.array(lat)
        self.XYZ = Earth.polarToCartesian(earthRadius, self.lon, self.lat)
        self.sharedPlateIds = sharedPlateIds
        self.boundType = 0

#Create child class for subduction plate boundaries
class SubductionBoundary(PlateBoundary):
    def __init__(self, sharedBound):
        
        #Run the parent's initialization
        PlateBoundary.__init__(self, sharedBound)
        
        #Get plate ids of overriding plate and subducting plates
        overPlateId, subPlateId = [], []
        for sharedSubSection in sharedBound.get_shared_sub_segments():
            overAndSubPlates = sharedSubSection.get_overriding_and_subducting_plates(True)
            if (overAndSubPlates != None):
                overridingPlate, subductingPlate, subduction_polarity = overAndSubPlates
                overPlateId.append(overridingPlate.get_feature().get_reconstruction_plate_id())
                subPlateId.append(subductingPlate.get_feature().get_reconstruction_plate_id())
        
        #Save data
        self.overPateId = np.unique(overPlateId)
        self.subPlateId = np.unique(subPlateId)
        self.boundType = 1

#Create child class for ridge plate boundaries
class RidgeBoundary(PlateBoundary):
    def __init__(self, sharedBound):
        PlateBoundary.__init__(self, sharedBound)
        self.boundType = 2

Now that we have defined our classes, we create a function that makes use of *pygplates* to return a list of plate boundary objects at a specified time.

In [ ]:
#Create function that returns a list of plate boundaries at specified times
def createPlateBoundariesAtTime(time,
                platePolygonsDirectory = './dataPygplates/Matthews_etal_GPC_2016_MesozoicCenozoic_PlateTopologies_PMAG.gpmlz',
                rotationsDirectory = './dataPygplates/Matthews_etal_GPC_2016_410-0Ma_GK07_PMAG.rot'):
    
    #Use pygplates to get shared boundary sections at specified time
    resolvedTopologies, sharedBoundarySections = [], []
    rotationModel = pygplates.RotationModel(rotationsDirectory)
    pygplates.resolve_topologies(platePolygonsDirectory, rotationModel, resolvedTopologies, time, sharedBoundarySections)
    
    #Loop through all shared plate boundaries
    plateBoundaries = []
    for sharedBound in sharedBoundarySections:

        #Identify which type of boundary this is
        boundType = sharedBound.get_feature().get_feature_type()
        isSubduction = boundType == pygplates.FeatureType.gpml_subduction_zone
        isOceanicRidge = boundType == pygplates.FeatureType.gpml_mid_ocean_ridge

        #Create plate boundary object of appropriate type and append to list of plate boundaries
        if isSubduction:
            plateBoundaries.append(SubductionBoundary(sharedBound))
        elif isOceanicRidge:
            plateBoundaries.append(RidgeBoundary(sharedBound))
        else:
            plateBoundaries.append(PlateBoundary(sharedBound))
            
    return plateBoundaries

The following function will be used for converting a list of plate boundary objects into a *pyvista* mesh of lines for better visualizations of the plate boundaries.

In [ ]:
def getBoundaryLines(plateBounds):
    
    #Variables used to create the line pyvista object
    XYZ, lineConnectivity, bType = [], [], []
    
    #Counter for keeping track of how many vertices we have used
    xyzCount = 0
    
    #Loop through all plate boundaries
    for bound in plateBounds:
        
        #Create lineID for defining line connectivity
        numOfPoints = len(bound.XYZ)
        lineConnectivity.append(numOfPoints)
        lineID = np.arange(numOfPoints) + xyzCount
        
        #Loop through points in plate boundary and append to arrays
        for i in range(numOfPoints):
            lineConnectivity.append(lineID[i])
            XYZ.append(bound.XYZ[i])
            bType.append(bound.boundType)
        xyzCount += numOfPoints
        
    #Create the line mesh
    lineMesh = pv.PolyData(np.array(XYZ), lines=lineConnectivity)
    lineMesh['boundType'] = np.array(bType)
    return lineMesh

pBounds = createPlateBoundariesAtTime(80)
lineMesh = getBoundaryLines(pBounds)

plotter = pv.PlotterITK()
plotter.add_mesh(pv.Sphere(radius=earthRadius-100))
plotter.add_mesh(lineMesh, scalars='boundType')
plotter.show()

# Distance from Plate Boundaries

To apply various tectonic forces, we will need to know how far a vertex is from plate boundaries. A simple yet naiive solution would be to use the *scipy.spatial.cKDTree()* to get the distances from sphere points to boundary points. Although this approach works well for sphere points which are fairly far from plate boundaries, it does not work well for points near the boundary regions, as demonstrated bellow. 

What we want to do instead, is to measure the distance from the line spanned by two boundary points.

In [ ]:
#Create a sphere
sphere = pv.Sphere(radius=earthRadius, theta_resolution=400, phi_resolution=400)
sphereXYZ = sphere.points

#Create plate boundaries and get its XYZ coordinates
plateBounds = createPlateBoundariesAtTime(0)
boundMesh = getBoundaryLines(plateBounds)
boundXYZ = boundMesh.points

#Calculate distances from points
distances = cKDTree(boundXYZ).query(sphereXYZ)[0]

#Display results
plotter = pv.PlotterITK()
plotter.add_mesh(sphere, scalars=distances**0.25)
plotter.add_mesh(boundMesh, scalars='boundType')
plotter.show()

We will now explain how we calculate a distance from a line with reference to the image bellow. Let $\textbf{s}$ be a point on the sphere, $\textbf{p}_1$ and $\textbf{p}_2$ be the two points spanning the boundary segments. In calculate the distance of $\textbf{s}$ from the line segment, there are 3 possible cases:

 - $\textbf{s}$ is closest to $\textbf{p}_1$
 - $\textbf{s}$ is closest to some point between $\textbf{p}_1$ and $\textbf{p}_2$
 - $\textbf{s}$ is closest to $\textbf{p}_2$
 
Let $\textbf{v} = \textbf{p}_2 - \textbf{p}_1$ and $\textbf{w} = \textbf{s} - \textbf{p}_1$, then we can check which case we are in with:

- Case 3: $\textbf{w} \cdot \textbf{v} <= 0$
- Case 1: If not case 3 and $\textbf{v} \cdot \textbf{v} <= \textbf{w} \cdot \textbf{v}$
- Case 2: Otherwise

Incase the reader may be unfamiliar with the dot product, taking the dot product of a vector with $\textbf{v}$ gives the magnitude of the projection of the vector onto $\textbf{v}$. Once we have identified which case we are in, we can make the appropriate distance calculation.

<div>
<img src="files/Images/ClosestToLineCropped2.png" width="600">
</div>

We split this task into 3 function and *getDistsToBounds()* will be the main function we call. It will use *prepareLines()* to get the centres and the points $\textbf{p}_1$ and $\textbf{p}_2$. We use *cKDTree()* to get the index of the closest line centres for each point on the sphere, which can be used to get the appropriate $\textbf{p}_1$ and $\textbf{p}_2$. We then use *getDistsToLinesSeg()* to calculate the appropriate distance as explained above. 

Since the function *getDistsToLinesSeg()* is slow, we use the *@jit(nopython=True)* decorator from the *numba* library to significantly speed things up. Although this decorator poses many restrictions on the content of the function such that we can not use many libraries apart from numpy, it speeds up our calculation by 10 times the speed. It is also a good habit to get into since it allows for GPU acceleration and can sometimes speed python code by 1000 times. If *numba* is not installed in your system, just uncomment the decorator and the function should run as normal, but slower.

In [ ]:
#Get line centres and points spanning each line
#Note: I could speed this up by making the plate boundary classes @jit aware, but this function is not slow to begin with
def prepareLines(plateBounds):
    lineCentres, linePoints = [], []
    for bound in plateBounds:
        for i in range(bound.XYZ.shape[0]-1):
            point1 = bound.XYZ[i]
            point2 = bound.XYZ[i+1]
            lineCentres.append((point1 + point2) / 2)
            linePoints.append([point1, point2])
    return np.array(lineCentres), np.array(linePoints)

#Get distance from line segments
@jit(nopython=True)
def getDistsToLinesSeg(sphereXYZ, closestLinePoints):
    distToBound = np.zeros(sphereXYZ.shape[0])
    for i in range(sphereXYZ.shape[0]):
        linePoints = closestLinePoints[i]
        xyz = sphereXYZ[i]

        #Append distance from vertex 0
        v = linePoints[1] - linePoints[0]
        w = xyz - linePoints[0]
        if np.dot(w, v) <= 0:
            distToZero = np.linalg.norm(linePoints[0] - xyz)
            distToBound[i] = distToZero

        #Append distance from vertex 1  
        elif np.dot(v, v) <= np.dot(w, v):
            distToOne = np.linalg.norm(linePoints[1] - xyz)
            distToBound[i] = distToOne

        #Append distance from somewhere in the line centre
        else:
            numerator = np.linalg.norm(np.cross(linePoints[1] - xyz, linePoints[1] - linePoints[0]))
            denominator = np.linalg.norm(linePoints[1] - linePoints[0])
            distToLine = numerator / denominator
            distToBound[i] = distToLine
    return distToBound

#Main function to call to get distance from plate boundaries
def getDistsToBounds(plateBounds, sphereXYZ):
    lineCentres, linePoints = prepareLines(plateBounds)
    distIds = cKDTree(lineCentres).query(sphereXYZ)[1]
    closestLinePoints = linePoints[distIds]
    distToBound = getDistsToLinesSeg(sphereXYZ, closestLinePoints)
    return distToBound

#Run the newly defined functions
distToBound = getDistsToBounds(plateBounds, sphereXYZ)

#Display results
plotter = pv.PlotterITK()
plotter.add_mesh(sphere, scalars=distToBound**0.25)
plotter.add_mesh(boundMesh, scalars='boundType')
plotter.show()

# Speed of Collision

When approximating forces such as subduction uplift or ocean floor formation, we will require the speed of plate collisions, and the speed of plate divergences.

In [ ]:
def getPlateCentres(sphereXYZ, plateIds):
    plateCentres = {}
    uniqueIds = np.unique(plateIds)
    for i in range(uniqueIds.shape[0]):
        plateXYZ = sphereXYZ[plateIds==uniqueIds[i]] * earthRadius
        plateCentres[uniqueIds[i]] = np.sum(plateXYZ, axis=0) / plateXYZ.shape[0]
    return plateCentres

In [ ]:
earth = Earth()
plateIds = earth.getPlateIdsAtTime(10)
rotations = earth.getRotations(plateIds, 10)

#Create a sphere
sphereXYZ = earth.sphereXYZ * earth.earthRadius
plateCentres = getPlateCentres(sphereXYZ, plateIds)

#Create plate boundaries and get its XYZ coordinates
plateBounds = createPlateBoundariesAtTime(0)
boundMesh = getBoundaryLines(plateBounds)
boundXYZ = boundMesh.points



In [ ]:
earth = Earth()
plateIds = earth.getPlateIdsAtTime(10)
rotations = earth.getRotations(plateIds, 10)
earthMesh = pv.PolyData(earth.sphereXYZ * earth.earthRadius, earth.earthFaces)

plateBounds = createPlateBoundariesAtTime(10)
rotationKeys = np.array(list(rotations.keys()))

boundXYZ, sharedId, boundPoints = [], [], []
for bound in plateBounds:
    for i in range(bound.XYZ.shape[0]-1):
        
        twoSharedIds = (len(bound.sharedPlateIds[i]) == 2)
        idsInKeys = np.in1d(bound.sharedPlateIds[i], rotationKeys)
        idsInKeys = np.all(idsInKeys)
        diffNotZero = np.all(bound.XYZ[i] - bound.XYZ[i+1] != 0)
        
        if twoSharedIds and idsInKeys and diffNotZero:
            sharedId.append(bound.sharedPlateIds[i])
            
            point1 = bound.XYZ[i]
            point2 = bound.XYZ[i+1]
            boundXYZ.append((point1 + point2) / 2)
            boundPoints.append([point1, point2])
            
sharedId = np.array(sharedId)
boundXYZ = np.array(boundXYZ)
boundPoints = np.array(boundPoints)

In [ ]:
sphereXYZ = earth.sphereXYZ * earth.earthRadius
plateCentres = getPlateCentres(sphereXYZ, plateIds)

speeds = []
for i in range(boundXYZ.shape[0]):
    xyz = boundXYZ[i]
    ids = sharedId[i]
    rot0 = rotations[ids[0]]
    rot1 = rotations[ids[1]]
    
    '''
    boundDirection = boundPoints[i, 1] - boundPoints[i, 0]
    perpendicularDirection = np.cross(boundDirection, xyz)
    perpendicularDirection = perpendicularDirection / np.linalg.norm(perpendicularDirection)
    '''
    
    movedXyz0 = rot0.apply(xyz)
    movedXyz1 = rot1.apply(xyz)
    
    cent0 = plateCentres[ids[0]]
    cent1 = plateCentres[ids[1]]
    centDist = np.linalg.norm(cent1 - cent0)
    centDistAfter = np.linalg.norm(rot1.apply(cent1) - rot0.apply(cent0))
    
    if centDist > centDistAfter:
        posOrNeg = 1
    else:
        posOrNeg = -1
    
    movedDist = np.sum((movedXyz1 - movedXyz0)**2)**0.5
    speeds.append(posOrNeg * movedDist)
speeds = np.array(speeds)

arrow = pv.Arrow(start=(-0.5, 0.0, 0.0))
boundMesh = pv.PolyData(boundXYZ * 1.05)
boundMesh['direction'] = boundXYZ
boundMesh['size'] = speeds
boundVectors = boundMesh.glyph(geom=arrow, orient='direction', scale='size', factor=1.0)

#Display results
plotter = pv.PlotterITK()
plotter.add_mesh(earthMesh, scalars=earth.heightHistory[-1])
plotter.add_mesh(boundVectors)
plotter.show()




            

            
sphereXYZ = earth.sphereXYZ



uniqueIds = np.unique(plateIds)

numOfPlates = uniqueIds.shape[0]
plateCentres = np.zeros((numOfPlates, 3))
vertexCount = np.zeros(numOfPlates)

for i in range(sphereXYZ.shape[0]):
    idx = np.argwhere(uniqueIds == plateIds[i])
    plateCentres[idx] += sphereXYZ[i]
    vertexCount[idx] += 1

plateCentres = (plateCentres.T / vertexCount).T
plateCentres *= earth.earthRadius

print(plateCentres)


sharedId = np.array(sharedId)
boundXYZ = np.array(boundXYZ)
boundPoints = np.array(boundPoints)

start = tme.time()
speeds = []
for i in range(boundXYZ.shape[0]):
    xyz = boundXYZ[i]
    ids = sharedId[i]
    rot0 = rotations[ids[0]]
    rot1 = rotations[ids[1]]
    
    boundDirection = boundPoints[i, 1] - boundPoints[i, 0]
    perpendicularDirection = np.cross(boundDirection, xyz)
    perpendicularDirection = perpendicularDirection / np.linalg.norm(perpendicularDirection)
    
    movedXyz0 = rot0.apply(xyz)
    movedXyz1 = rot1.apply(xyz)
    
    #This is wrong... Rotations are not given in any particular order
    if np.dot(movedXyz0, perpendicularDirection) >= 0:
        posOrNeg = 1
    else:
        posOrNeg = -1
    
    movedDist = np.sum((movedXyz1 - movedXyz0)**2)**0.5
    speeds.append(posOrNeg * movedDist)
speeds = np.array(speeds)
print(tme.time() - start)


arrow = pv.Arrow(start=(-0.5, 0.0, 0.0))
boundMesh = pv.PolyData(boundXYZ * 1.05)
boundMesh['direction'] = boundXYZ
boundMesh['size'] = speeds
boundVectors = boundMesh.glyph(geom=arrow, orient='direction', scale='size', factor=1.0)

#Display results
plotter = pv.PlotterITK()
plotter.add_mesh(earthMesh, scalars=earth.heightHistory[-1])
plotter.add_mesh(boundVectors)
plotter.show()

#Create a sphere
sphere = pv.Sphere(radius=earthRadius, theta_resolution=400, phi_resolution=400)
sphereXYZ = sphere.points

#Create plate boundaries and get its XYZ coordinates
plateBounds = createPlateBoundariesAtTime(0)
boundMesh = getBoundaryLines(plateBounds)
boundXYZ = boundMesh.points



#rotations = Earth.getRotations(plateIds, 5, 5, rotationModel)


In [ ]:
'''
start = tme.time()
distToBound = []
for i in range(sphereXYZ.shape[0]):
    linePoints = closestLinePoints[i]
    xyz = sphereXYZ[i]
    
    #Append distance from vertex 0
    v = linePoints[1] - linePoints[0]
    w = xyz - linePoints[0]
    if np.dot(w, v) <= 0:
        distToZero = np.linalg.norm(linePoints[0] - xyz)
        distToBound.append(distToZero)

    #Append distance from vertex 1  
    elif np.dot(v, v) <= np.dot(w, v):
        distToOne = np.linalg.norm(linePoints[1] - xyz)
        distToBound.append(distToOne)

    #Append distance from somewhere in the line centre
    else:
        numerator = np.linalg.norm(np.cross(linePoints[1] - xyz, linePoints[1] - linePoints[0]))
        denominator = np.linalg.norm(linePoints[1] - linePoints[0])
        distToLine = numerator / denominator
        distToBound.append(distToLine)

print(tme.time() - start)

distToBound = np.array(distToBound)
print(closestLinePoints.shape)
print(sphereXYZ.shape)
'''

lineCentres, linePoints = [], []
for bound in plateBounds:
    numOfPoints = bound.XYZ.shape[0]
    for i in range(numOfPoints-1):
        point1 = bound.XYZ[i]
        point2 = bound.XYZ[i+1]
        lineCentres.append((point1 + point2) / 2)
        linePoints.append([point1, point2])

linePoints = np.array(linePoints)
distIds = cKDTree(lineCentres).query(sphereXYZ)[1]
closestLinePoints = linePoints[distIds]

start = tme.time()
distToBound = []
for i in range(sphereXYZ.shape[0]):
    linePoints = closestLinePoints[i]
    xyz = sphereXYZ[i]
    
    #Append distance from vertex 0
    v = linePoints[1] - linePoints[0]
    w = xyz - linePoints[0]
    if np.dot(w, v) <= 0:
        distToZero = np.linalg.norm(linePoints[0] - xyz)
        distToBound.append(distToZero)

    #Append distance from vertex 1  
    elif np.dot(v, v) <= np.dot(w, v):
        distToOne = np.linalg.norm(linePoints[1] - xyz)
        distToBound.append(distToOne)

    #Append distance from somewhere in the line centre
    else:
        numerator = np.linalg.norm(np.cross(linePoints[1] - xyz, linePoints[1] - linePoints[0]))
        denominator = np.linalg.norm(linePoints[1] - linePoints[0])
        distToLine = numerator / denominator
        distToBound.append(distToLine)

print(tme.time() - start)

distToBound = np.array(distToBound)
print(closestLinePoints.shape)
print(sphereXYZ.shape)


#Display results
plotter = pv.PlotterITK()
plotter.add_mesh(sphere, scalars=distToBound**0.25)
plotter.add_mesh(boundMesh, scalars='boundType')
plotter.show()


from numba import jit



pBounds = createPlateBoundariesAtTime(0)

idx = 0
XYZ, line, bType = [], [], []
for bound in pBounds:
    line.append(len(bound.XYZ))
    lineID = np.arange(len(bound.XYZ)) + idx
    
    for i in lineID:
        line.append(i)
    idx += len(bound.XYZ)
    
    for xyz in bound.XYZ:
        XYZ.append(xyz)
        bType.append(bound.boundType)


XYZ = pv.PolyData(np.array(XYZ), lines=line)
bType = np.array(bType)

plotter = pv.PlotterITK()
plotter.add_mesh(pv.Sphere(radius=earthRadius-100))
plotter.add_mesh(XYZ, scalars=bType)
plotter.show()

sphere = pv.Sphere()
plotter = pv.PlotterITK()
plotter.add_mesh(sphere)
plotter.show()

#Get pygplates to resolve data at specified time
resolvedTopologies, sharedBoundarySections = [], []
pygplates.resolve_topologies(platePolygonsDirectory, rotationModel, resolvedTopologies, time, sharedBoundarySections)

plateBoundaries = []

#Loop through all shared plate boundaries
for sharedBound in sharedBoundarySections:
    
    #Identify which type of boundary this is
    boundType = sharedBound.get_feature().get_feature_type()
    isSubduction = boundType == pygplates.FeatureType.gpml_subduction_zone
    isOceanicRidge = boundType == pygplates.FeatureType.gpml_mid_ocean_ridge
    
    #Create plate boundary object of appropriate type and append to list of plate boundaries
    if isSubduction:
        plateBoundaries.append(SubductionBoundary(sharedBound))
    elif isOceanicRidge:
        plateBoundaries.append(RidgeBoundary(sharedBound))
    else:
        plateBoundaries.append(PlateBoundary(sharedBound))

plt.figure(figsize=(12, 8))
for bound in plateBoundaries:
    if bound.boundType == 'ridge':
        plt.plot(bound.lon, bound.lat)
plt.show()